# Fast Batch Multitask Net GP

In [1]:
import fastgp
import torch
import numpy as np

In [2]:
torch.set_default_dtype(torch.float64)

## True Function

In [3]:
def f(l,x):
    weights = 2**torch.arange(1,l+2)
    return torch.vstack([
        (torch.sin(weights*np.pi*x)/weights).sum(1),
        (torch.cos(weights*np.pi*x)/weights).sum(1),
    ])
num_tasks = 4
d = 1 # dimension
rng = torch.Generator().manual_seed(17)
x = torch.rand((2**7,d),generator=rng) # random testing locations
y = torch.cat([f(l,x)[:,None,:] for l in range(num_tasks)],1) # true values at random testing locations
z = torch.rand((2**8,d),generator=rng) # other random locations at which to evaluate covariance
print("x.shape = %s"%str(tuple(x.shape)))
print("y.shape = %s"%str(tuple(y.shape)))
print("z.shape = %s"%str(tuple(z.shape)))

x.shape = (128, 1)
y.shape = (2, 4, 128)
z.shape = (256, 1)


## Construct Fast GP

In [4]:
fgp = fastgp.FastGPDigitalNetB2(d,seed_for_seq=7,num_tasks=num_tasks,shape_batch=[2,])
x_next = fgp.get_x_next(n=2**torch.arange(4,4+num_tasks))
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
fgp.add_y_next(y_next)
assert len(x_next)==len(y_next)
for i in range(len(x_next)):
    print("i = %d"%i)
    print("\tx_next[%d].shape = %s"%(i,str(tuple(x_next[i].shape))))
    print("\ty_next[%d].shape = %s"%(i,str(tuple(y_next[i].shape))))

i = 0
	x_next[0].shape = (16, 1)
	y_next[0].shape = (2, 16)
i = 1
	x_next[1].shape = (32, 1)
	y_next[1].shape = (2, 32)
i = 2
	x_next[2].shape = (64, 1)
	y_next[2].shape = (2, 64)
i = 3
	x_next[3].shape = (128, 1)
	y_next[3].shape = (2, 128)


In [5]:
pmean = fgp.post_mean(x)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-pmean,dim=-1)/torch.linalg.norm(y,dim=-1)))

pmean.shape = (2, 4, 128)
l2 relative error:
tensor([[0.1521, 0.0732, 0.0379, 0.0384],
        [0.1829, 0.0839, 0.0448, 0.0554]])


In [6]:
data = fgp.fit()
list(data.keys())

     iter of 5.0e+03 | NMLL       | norm term  | logdet term
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            0.00e+00 | 4.78e+02   | 7.77e+00   | -4.12e+02 
            5.00e+00 | -6.52e+02  | 1.57e+02   | -1.69e+03 
            1.00e+01 | -9.28e+02  | 2.82e+02   | -2.09e+03 
            1.50e+01 | -1.04e+03  | 4.64e+02   | -2.38e+03 
            2.00e+01 | -1.10e+03  | 4.71e+02   | -2.46e+03 
            2.50e+01 | -1.14e+03  | 4.85e+02   | -2.51e+03 
            3.00e+01 | -1.17e+03  | 4.92e+02   | -2.55e+03 
            3.50e+01 | -1.18e+03  | 4.84e+02   | -2.55e+03 
            4.00e+01 | -1.18e+03  | 4.82e+02   | -2.55e+03 
            4.50e+01 | -1.18e+03  | 4.73e+02   | -2.54e+03 
            5.00e+01 | -1.18e+03  | 4.75e+02   | -2.54e+03 
            5.50e+01 | -1.18e+03  | 4.77e+02   | -2.54e+03 
            6.00e+01 | -1.19e+03  | 4.78e+02   | -2.55e+03 
            6.50e+01 | -1.19e+03  | 4.78e+02   | -2.55e+03 
            7.00e+01 | -1.19e+03  | 4.

['mll_hist', 'scale_hist', 'lengthscales_hist', 'task_kernel_hist']

In [7]:
pmean,pvar,q,ci_low,ci_high = fgp.post_ci(x,confidence=0.99)
print("pmean.shape = %s"%str(tuple(pmean.shape)))
print("pvar.shape = %s"%str(tuple(pvar.shape)))
print("q = %.2f"%q)
print("ci_low.shape = %s"%str(tuple(ci_low.shape)))
print("ci_high.shape = %s"%str(tuple(ci_high.shape)))
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-pmean,dim=-1)/torch.linalg.norm(y,dim=-1)))
pcov = fgp.post_cov(x,x)
print("pcov.shape = %s"%str(tuple(pcov.shape)))
_range0,_rangen1 = torch.arange(pcov.size(0)),torch.arange(pcov.size(-1))
assert torch.allclose(pcov[_range0,_range0][:,_rangen1,_rangen1],pvar) and (pvar>=0).all()
pcov2 = fgp.post_cov(x,z)
print("pcov2.shape = %s"%str(tuple(pcov2.shape)))

pmean.shape = (2, 4, 128)
pvar.shape = (4, 128)
q = 2.58
ci_low.shape = (2, 4, 128)
ci_high.shape = (2, 4, 128)
l2 relative error:
tensor([[0.1350, 0.0766, 0.0542, 0.0448],
        [0.1758, 0.0942, 0.0681, 0.0656]])
pcov.shape = (4, 4, 128, 128)
pcov2.shape = (4, 4, 128, 256)


In [8]:
pcmean,pcvar,q,cci_low,cci_high = fgp.post_cubature_ci(confidence=0.99)
print("pcmean:\n%s"%str(pcmean))
print("\npcvar:\n%s"%str(pcvar))
print("\ncci_low:\n%s"%str(cci_low))
print("\ncci_high:\n%s"%str(cci_high))

pcmean:
tensor([[ 9.6223e-19,  3.7947e-19,  4.8789e-19,  6.7763e-19],
        [-4.7976e-18, -6.6678e-18, -1.0354e-17, -9.7578e-18]])

pcvar:
tensor([1.6881e-05, 1.4482e-05, 1.2366e-05, 1.2779e-05])

cci_low:
tensor([[-0.0106, -0.0098, -0.0091, -0.0092],
        [-0.0106, -0.0098, -0.0091, -0.0092]])

cci_high:
tensor([[0.0106, 0.0098, 0.0091, 0.0092],
        [0.0106, 0.0098, 0.0091, 0.0092]])


## Project and Increase Sample Size

In [9]:
n_new = fgp.n*2**torch.arange(num_tasks-1,-1,-1)
pcov_future = fgp.post_cov(x,z,n=n_new)
pvar_future = fgp.post_var(x,n=n_new)
pcvar_future = fgp.post_cubature_var(n=n_new)

In [10]:
x_next = fgp.get_x_next(n_new)
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
for _y in y_next:
    print(_y.shape)
fgp.add_y_next(y_next)
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-fgp.post_mean(x),dim=-1)/torch.linalg.norm(y,dim=-1)))
assert torch.allclose(fgp.post_cov(x,z),pcov_future)
assert torch.allclose(fgp.post_var(x),pvar_future)
assert torch.allclose(fgp.post_cubature_var(),pcvar_future)

torch.Size([2, 112])
torch.Size([2, 96])
torch.Size([2, 64])
torch.Size([2, 0])
l2 relative error:
tensor([[0.0386, 0.0370, 0.0470, 0.0491],
        [0.0403, 0.0364, 0.0603, 0.0727]])


In [11]:
data = fgp.fit(verbose=False,store_mll_hist=False,store_scale_hist=False,store_lengthscales_hist=False,store_noise_hist=False)
print("l2 relative error:\n%s"%str(torch.linalg.norm(y-fgp.post_mean(x),dim=-1)/torch.linalg.norm(y,dim=-1)))

l2 relative error:
tensor([[0.0343, 0.0365, 0.0317, 0.0400],
        [0.0343, 0.0355, 0.0520, 0.0677]])


In [12]:
n_new = fgp.n*2**torch.arange(num_tasks)
pcov_new = fgp.post_cov(x,z,n=n_new)
pvar_new = fgp.post_var(x,n=n_new)
pcvar_new = fgp.post_cubature_var(n=n_new)
x_next = fgp.get_x_next(n_new)
y_next = [f(l,x_next[l]) for l in range(num_tasks)]
fgp.add_y_next(y_next)
assert torch.allclose(fgp.post_cov(x,z),pcov_new)
assert torch.allclose(fgp.post_var(x),pvar_new)
assert torch.allclose(fgp.post_cubature_var(),pcvar_new)